In [0]:
# The code was removed by Watson Studio for sharing.

In [0]:
#dd-ignore

!pip install --user dd-scenario


In [0]:
#dd-ignore

from dd_scenario import *

# Creates a client...
# If you want to be able to call solve() on the client, you have to provide your API Key
# client = Client(pc=pc, apikey='IAM_APIKEY')
client = Client(pc=pc)


In [0]:
#dd-ignore

#Get 'ResourceAllocation' decision...
dd_model_builder = client.get_model_builder(name="ResourceAllocation")

#Get scenario 'Shipment_MICounty'...
scenario = dd_model_builder.get_scenario(name="Shipment_MICounty")

#Load all input data as a map { data_name: data_frame }
inputs = scenario.get_tables_data(category='input')
# This will hold all outputs as a map { data_name: data_frame }
outputs = {}

# we use a lock to access ``outputs``. This allows solves() to
# be aborted without race condition in data writting
import threading
output_lock = threading.Lock()



In [0]:
from docplex.mp.model import Model
import pandas 

# Load CSV files into inputs dictionnary
inputs = get_all_inputs()
markets = inputs['LedgerToWork']
markets.head()

# We don't want the markets to have more material than their forecasted demand.  So capping the max ship
markets['MAX_SHIP'] = markets['FCST_DEMAND_1_WK'] - markets['CURR_INV']
markets.loc[markets['MAX_SHIP'] < 0, 'MAX_SHIP'] = 0

# Total cases avaivlale for shipment is hardcoded for now.  Should really come from an external source
total_shipments = inputs['shipments']
total_shipments = total_shipments.iloc[0,0]

# Total current inventory, total demand and an overall optimum allocation ratio across all markets
total_curr_inv = markets.CURR_INV.sum()
total_demand = markets.FCST_DEMAND_1_WK.sum()
opt_allocation_ratio = (total_shipments + total_curr_inv)/total_demand

# Model
mdl = Model(name='prs4b')

# Create integer decision variables for shipments.  Lower bound is 0.  Upper bound is forecast - curr inv
qty = markets.copy()
qty['VAR'] = qty.apply(lambda x: mdl.integer_var(name=x['CITY'], ub = x['MAX_SHIP'], lb = 0), axis=1)
# make the name the index
qty.set_index('CITY', inplace=True)

# Create a dummy decision variable representing the diff in allocation ratio.
# Need this dummy dvs to account for the fact that we need to minimize the sum of the absolute
# differences between the allocation ratio in individual markets.  The presence of the 
# absolute number in the objective will make it non-linear. 
# If we have 25 markets, we will have 24 dvs.  DV for market 1 -> 2, DV for market 1 -> 3, 
# market 1 -> 3, etc to 1 - > 25
dummy_dvs = pandas.DataFrame({'CITY_A': [qty.index.values[0]]*(qty.shape[0])})
dummy_dvs['CITY_B'] = qty.index.values
dummy_dvs = dummy_dvs.loc[dummy_dvs['CITY_A'] != dummy_dvs['CITY_B']]
dummy_dvs['IDX'] = 'DV_' + dummy_dvs['CITY_A'] + '_' + dummy_dvs['CITY_B']
dummy_dvs['VAR'] = dummy_dvs.apply(lambda x: mdl.continuous_var(name = 'DV_' + x['CITY_A'] + '_' + x['CITY_B']), axis=1)
dummy_dvs.set_index('IDX', inplace = True)
dummy_dvs.head()

# Add constraints
# Total shipment across all markets cannot exceed total available product from suppier
mdl.add_constraint(mdl.sum(qty.loc[m.CITY]['VAR'] for m in markets.itertuples())<= total_shipments, 'CT_Total_Shipment')
mdl.add_constraints( \
                    (((mdl.sum([qty.loc[d[1]]['VAR'], qty.loc[d[1]]['CURR_INV']]))/qty.loc[d[1]]['FCST_DEMAND_1_WK'] - \
                      (mdl.sum([qty.loc[d[2]]['VAR'], qty.loc[d[2]]['CURR_INV']]))/qty.loc[d[2]]['FCST_DEMAND_1_WK']) \
                     <= d[3]) for d in dummy_dvs.itertuples())
mdl.add_constraints( \
                    (-1*((mdl.sum([qty.loc[d[1]]['VAR'], qty.loc[d[1]]['CURR_INV']]))/qty.loc[d[1]]['FCST_DEMAND_1_WK'] - \
                      (mdl.sum([qty.loc[d[2]]['VAR'], qty.loc[d[2]]['CURR_INV']]))/qty.loc[d[2]]['FCST_DEMAND_1_WK']) \
                     <= d[3]) for d in dummy_dvs.itertuples())
                     
#Defining the KPIs
for n in markets.itertuples():
    shortage_ratio_with_var = ((qty.loc[n.CITY]['FCST_DEMAND_1_WK']-(qty.loc[n.CITY]['CURR_INV']))-qty.loc[n.CITY]['VAR'])/qty.loc[n.CITY]['FCST_DEMAND_1_WK']
    
    mdl.add_kpi(shortage_ratio_with_var, publish_name='Shortage percentage for %s' % n.CITY)

# Define the objective function 
total_ratio = mdl.sum((-1*(qty.loc[m.CITY]['VAR'] + m.CURR_INV)/m.FCST_DEMAND_1_WK) for m in markets.itertuples())
total_dummy_vars = mdl.sum((d[3]) for d in dummy_dvs.itertuples())
mdl.add_kpi(-1*(total_ratio/25), publish_name="Total Allocation Ratio")
mdl.add_kpi(total_dummy_vars, publish_name="Total Shortage Ratio")
mdl.minimize(total_ratio + total_dummy_vars)


mdl.print_information()

# solve
ok = mdl.solve()
mdl.print_solution()
mdl.report()

#Creating the output variable filled up with model solution 
import pandas
import numpy

solution_df = pandas.DataFrame(columns=['name', 'value'])

for index, dvar in enumerate(mdl.solution.iter_variables()):
    if(not dvar.to_string().startswith('DV_')):
        solution_df.loc[index,'name'] = dvar.to_string()
        solution_df.loc[index,'value'] = dvar.solution_value


In [0]:
solution_df


In [0]:
outputs['solution'] = solution_df